In [1]:
import pandas as pd
import xlsxwriter
import json
import requests
import datetime
from IPython.display import JSON

In [2]:
edition = {'en': 'MAIN/2019-07-31',
           'es': 'MAIN/SNOMEDCT-ES/2019-10-31',
           'nl': 'MAIN/SNOMEDCT-NL/2019-09-30',
           'fr': 'MAIN/SNOMEDCT-BE/2020-03-15',
           'da': 'MAIN/SNOMEDCT-DK/2020-03-31'}

def getUri(lang):
    return 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/browser/' + edition[lang] + '/concepts/'

def getConcept(sctid, lang):
    uri = getUri(lang)
    resp = requests.get(uri + str(sctid))
    return resp.json()
getUri('nl')

'https://browser.ihtsdotools.org/snowstorm/snomed-ct/browser/MAIN/SNOMEDCT-NL/2019-09-30/concepts/'

In [18]:
concept = getConcept('73211009', 'nl')
JSON(concept)

<IPython.core.display.JSON object>

In [19]:
[('FSN' if desc['type'] == 'FSN' else list(desc['acceptabilityMap'].values())[0], desc['term']) for desc in concept['descriptions'] if (desc['active'] == True and desc['lang'] == 'nl')]

[('PREFERRED', 'diabetes mellitus'),
 ('FSN', 'diabetes mellitus (aandoening)'),
 ('PREFERRED', 'diabetes'),
 ('ACCEPTABLE', 'suikerziekte')]

In [17]:
# Select active descriptions of language, returh FSN if there is one, otherwise PREFERRED, else a SYNONYM
def getName(sctid, lang):
    concept = getConcept(sctid, lang)
    descs = [desc for desc in concept['descriptions'] if (desc['active'] == True and desc['lang'] == lang)]
    term = ''
    for desc in descs:
        if desc['type'] == 'FSN':
            return desc['term']
        # This ignores multiple map items which happens with en-US and en-GB, to solve this we'd need to include the lang refset id
        elif list(desc['acceptabilityMap'].values())[0] == 'PREFERRED':
            term = desc['term']
        elif term == '':
            term = desc['term']
    return term

getName('112144000', 'nl')

'bloedgroep A (bevinding)'

In [93]:
# prio1 vorig
dsuri = 'https://decor.nictiz.nl/decor/services/RetrieveTransaction?id=2.16.840.1.113883.2.4.3.11.60.90.77.4.2499&effectiveDate=2020-01-08T13:47:27&language=nl-NL&format=json'
#prio 1 huidig
# dsuri = 'https://decor.nictiz.nl/decor/services/RetrieveTransaction?id=2.16.840.1.113883.2.4.3.11.60.90.77.4.2493&effectiveDate=2020-01-08T13:13:15&language=nl-NL&format=json'
resp = requests.get(dsuri)
resp.status_code

200

In [94]:
data = resp.json()
dataset = data["dataset"][0]
name = dataset.get('shortName', 'sheet')
JSON(dataset)

<IPython.core.display.JSON object>

In [97]:
def makeRows(parent, codes):
    for concept in parent["concept"]:
        if 'valueSet' in concept.keys():
            if ('name' in concept['valueSet'][0]):
                for code in concept['valueSet'][0]['conceptList'][0]['concept']:
                    codes.append(code)
        if 'terminologyAssociation' in concept.keys():
            for ta in concept['terminologyAssociation']:
                codes.append(ta)
        if 'concept' in concept.keys():
            makeRows(concept, codes)
    return

codes = []
df = makeRows(dataset, codes)
df = pd.DataFrame(codes)

In [98]:
df = df[['code', 'codeSystem', 'displayName']]
df.head(20)

,code,codeSystem,displayName
0,Z3,2.16.840.1.113883.2.4.15.1060,Huisartsenpraktijk
1,G3,2.16.840.1.113883.2.4.15.1060,Verloskundigenpraktijk
2,V4,2.16.840.1.113883.2.4.15.1060,Ziekenhuis
3,B2,2.16.840.1.113883.2.4.15.1060,Echocentrum
4,L1,2.16.840.1.113883.2.4.15.1060,Laboratorium
5,NaN,NaN,NaN
6,Z3,2.16.840.1.113883.2.4.15.1060,Huisartsenpraktijk
7,G3,2.16.840.1.113883.2.4.15.1060,Verloskundigenpraktijk
8,V4,2.16.840.1.113883.2.4.15.1060,Ziekenhuis
9,B2,2.16.840.1.113883.2.4.15.1060,Echocentrum


In [99]:
def getCode(row, lang):
    if (row['codeSystem'] == '2.16.840.1.113883.6.96'):
        try:
            return getName(row['code'], lang)
        except:
            pass

In [100]:
for lang in ['nl', 'en', 'es', 'fr', 'da']:
    df[lang] = df.apply(lambda row: getCode(row, lang), axis=1)
    print('Done: ', lang)

Done:  nl
Done:  en
Done:  es
Done:  fr
Done:  da


In [101]:
df

,code,codeSystem,displayName,nl,en,es,fr,da
0,Z3,2.16.840.1.113883.2.4.15.1060,Huisartsenpraktijk,None,None,None,None,None
1,G3,2.16.840.1.113883.2.4.15.1060,Verloskundigenpraktijk,None,None,None,None,None
2,V4,2.16.840.1.113883.2.4.15.1060,Ziekenhuis,None,None,None,None,None
3,B2,2.16.840.1.113883.2.4.15.1060,Echocentrum,None,None,None,None,None
4,L1,2.16.840.1.113883.2.4.15.1060,Laboratorium,None,None,None,None,None
...,...,...,...,...,...,...,...,...
103,359940006,2.16.840.1.113883.6.96,Partial breech extraction (procedure),extractie van foetus in half volkomen stuitlig...,Partial breech extraction (procedure),extracción de nalgas parcial (procedimiento),,Partiel sædeekstraktion
104,416055001,2.16.840.1.113883.6.96,Total breech extraction (procedure),stuitextractie van foetus in volkomen stuitlig...,Total breech extraction (procedure),extracción de nalgas total (procedimiento),,fuldstændig sædeekstraktion
105,3311000146109,2.16.840.1.113883.6.96,Assisted delivery of fetus (procedure),geassisteerde vaginale verlossing (verrichting),None,None,None,None
106,9274-2,2.16.840.1.113883.6.1,Apgarscore na 5 min.,None,None,None,None,None


In [103]:
writer = pd.ExcelWriter('prio1-vorig-trans.xlsx')
df.to_excel(writer)
writer.save()